In [ ]:
import numpy as np
import PIL
import matplotlib.pyplot as plt

image = PIL.Image.open('./images/child.jpg')
image = image.convert("RGB")
img = np.array(image, dtype=np.uint8)

In [ ]:
def change_brightness(img, brightness):
    brightness = np.clip(int(brightness), -255, 255)
    new_image = np.clip(img.astype(np.uint16) + brightness, 0, 255)
    return new_image.astype(np.uint8)

In [ ]:
def change_contrast(img, contrast):
    contrast = np.clip(float(contrast), -255, 255)
    factor = (259 * (contrast + 255)) / (255 * (259 - contrast))
    new_image = np.clip(factor * (img.astype(float) - 128) + 128, 0, 255)
    return new_image.astype(np.uint8)

In [ ]:
def flip_image(img, dir=1):
    if dir == 1:
        return np.flip(img, axis=0).astype(np.uint8)
    else:
        return np.flip(img, axis=1).astype(np.uint8)

In [ ]:
def grayscale_image(img):
    return np.dot(img[..., :3], [0.3, 0.59, 0.11]).astype(np.uint8)

In [ ]:
def sepia_image(img):
    sepia_matrix = np.array([[0.393, 0.769, 0.189],
                            [0.349, 0.686, 0.168],
                            [0.272, 0.534, 0.131]])
    return np.clip(np.dot(img, sepia_matrix.T), 0, 255).astype(np.uint8)

In [ ]:
def blur_image(img, size, sigma):
    kernel = kernel_1d(size, sigma)
    result = np.ones_like(img, dtype=np.float64) * 255
    
    if(img.shape[-1] != 3):
        result[:, :] = convolution_channel(img[:,:], kernel)
    else:
        for c in range(img.shape[-1]):
            result[:, :, c] = convolution_channel(img[:,:,c], kernel)

    return np.uint8(result)

def kernel_1d(size, sigma):
    x = np.linspace(-(size // 2), size // 2, size)
    kernel = (1 / (np.sqrt(2 * np.pi) * sigma**2)) * np.exp(-x**2 / (2 * sigma**2))
    kernel /= np.sum(kernel)
    return kernel

def convolution_channel(channel, kernel):   
    result = np.apply_along_axis(lambda x: np.convolve(x, kernel, mode='same'), axis=0, arr=channel)
    result = np.apply_along_axis(lambda x: np.convolve(x, kernel, mode='same'), axis=1, arr=result)
    return result

In [ ]:
def sharp_image(img, size, sigma):
    identity = np.zeros(shape=size)
    identity[size//2] = 1
    kernel = 2*identity - kernel_1d(size, sigma)
    result = np.ones_like(img, dtype=np.float64) * 255
    
    if(img.shape[-1] != 3):
        result[:, :] = convolution_channel(img[:,:], kernel)
    else:
        for c in range(img.shape[-1]):
            result[:, :, c] = convolution_channel(img[:,:,c], kernel)
        
    result = np.clip(result, 0, 255)
    return np.uint8(result)

def kernel_1d(size, sigma):
    x = np.linspace(-(size // 2), size // 2, size)
    kernel = np.exp(-x**2 / (2 * sigma**2))
    kernel /= np.sum(kernel)
    return kernel

def convolution_channel(channel, kernel):   
    result = np.apply_along_axis(lambda x: np.convolve(x, kernel, mode='same'), axis=0, arr=channel)
    result = np.apply_along_axis(lambda x: np.convolve(x, kernel, mode='same'), axis=1, arr=result)
    return result

In [ ]:
def zoom_image(img, percentage):
    new_height = int(img.shape[0] * percentage)
    new_width = int(img.shape[1] * percentage)
    
    center_row = img.shape[0] // 2
    center_col = img.shape[1] // 2
    
    col_start = center_col - new_width // 2
    col_end = col_start + new_width
    
    row_start = center_row - new_height // 2
    row_end = row_start + new_height
    
    return img[row_start:row_end, col_start:col_end, :]

In [ ]:
def crop_circle(img):
    center_x = img.shape[1] // 2
    center_y = img.shape[0] // 2
    y, x = np.ogrid[:img.shape[0], :img.shape[1]]
    
    radius = min(center_x, center_y)
        
    mask = ((x - center_x)**2 + (y - center_y)**2) <= radius**2
    cropped_image = img.copy()
    cropped_image[~mask] = 0    
    return cropped_image

In [ ]:
def crop_butterfly(img, elipse_size=8):
    center_x = img.shape[1] // 2
    center_y = img.shape[0] // 2

    elipse = (img.shape[1] / elipse_size, 
              img.shape[0] / elipse_size, 
              img.shape[1] - img.shape[1] / elipse_size, 
              img.shape[0] - img.shape[0] / elipse_size)
    
    a = np.linalg.norm(np.array([elipse[0], elipse[1]]) - np.array([elipse[2], -elipse[3]])) / 2
    b = np.sqrt(a**2 - (np.linalg.norm(np.array([elipse[0], elipse[1]]) - np.array([elipse[2], elipse[3]])) / 2)**2)
    
    y, x = np.ogrid[:img.shape[0], :img.shape[1]]
    ellipse_rotation = 45
    x_rot = (x - center_x) * np.cos(np.radians(ellipse_rotation)) - (y - center_y) * np.sin(np.radians(ellipse_rotation))
    y_rot = (x - center_x) * np.sin(np.radians(ellipse_rotation)) + (y - center_y) * np.cos(np.radians(ellipse_rotation))
    
    mask1 = (x_rot)**2 / a**2 + (y_rot) **2 / b**2 <= 1
    mask2 = np.flip(mask1, axis=0)
    mask = np.logical_or(mask1, mask2)
    cropped_img = img.copy()
    cropped_img[~mask] = 0
    return cropped_img

In [ ]:
new_img = crop_butterfly(img, 8)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

axes[0].imshow(img)
axes[0].axis('off')
axes[0].set_title('Before')

axes[1].imshow(new_img, cmap='gray')
axes[1].axis('off')
axes[1].set_title('After')

plt.tight_layout()
plt.show()

In [ ]:
import os

output_img = PIL.Image.fromarray(new_img)

output_img.save('./output/new_img.jpg')